# Finding Optimum XGBoost Parameters for Tabular For Homesite Competition
> Building on the work of the previous notebook, which referenced [Zach's notebook](https://github.com/muellerzr/Practical-Deep-Learning-for-Coders-2.0/blob/master/Tabular%20Notebooks/02_Ensembling.ipynb) to apply his techniques for permutation importance and ensemble learning to the Homesite Competition problem set, we will deep dive into optimizing the XGBoost parameters to see if an improved model can be generated, and what effect this has for our resulting ensemble model's predictions

- toc: true 
- badges: true
- comments: true
- categories: [kaggle, fastai]
- author: Nissan Dookeran
- image: images/chart-preview.png

## Introduction
Using the [code here](https://machinelearningmastery.com/tune-number-size-decision-trees-xgboost-python/) we will look at the optimal number of estimators, depth, and then combination of the two when using XGBoost to generate a model

Notes:
- Possible improvements if manually coding would be to add `subsamples` as a variable, however with both the increase of permutations and the long running times for GridSearch and a look to explore other options like Optuna and Bayesian Search to automate this, I limited it to just `max_depth` and `learning_rate` as the additional variables for now
- Because of timeouts for long ranges, broke up the ranges for the various parameters into groupings, will only explore a subsequent grouping if there is an optimal value that is max in a previous grouping. This will also give a more manageable range for the permutations when trying to find the best combination of all parameters 
- Added to tune learning_rate based on [this code](https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/)
- Removed the `triage` variable, `FillMissing` will handle NA values in categorical fields appropriately, but just need to filter it out from modifying categories function
- Changed the categorize functions from [last notebook](https://redditech.github.io/team-fast-tabulous/kaggle/fastai/2021/06/27/Improving-Fastai-split-choices.html) to exclude any columns in y_names from being evaluated since these shouldn't be part of the model training as a parameter
- Added code to save all models to reuse in another notebook that submits to Kaggle for test evaluation

## Setup

>> Adding based on [tutorial](https://github.com/fastai/course-v3/blob/master/nbs/dl1/00_notebook_tutorial.ipynb) for notebooks

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
!pip install -Uqq fastai
!pip install kaggle
from fastai.tabular.all import *

     |████████████████████████████████| 194kB 14.1MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 


In [ ]:
global gdrive #colab only code block
gdrive = Path('/content/gdrive/My Drive')
from google.colab import drive
if not gdrive.exists(): drive.mount(str(gdrive.parent))
!mkdir -p ~/.kaggle
!cp /content/gdrive/MyDrive/Kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/gdrive


In [ ]:
from kaggle import api
path = Path.cwd()
path.ls()

(#3) [Path('/content/.config'),Path('/content/gdrive'),Path('/content/sample_data')]

> Only run the next three lines the first time if in a local repository. This will prevent large training data files and model files being checked into Github

In [ ]:
!touch .gitignore

In [ ]:
!echo "_data" > .gitignore

In [ ]:
!mkdir _data

In [ ]:
os.chdir('_data')
Path.cwd()

Path('/content/_data')

> Back to it

In [ ]:
os.chdir(path/"gdrive/MyDrive/Kaggle/") # colab only code
Path.cwd()

Path('/content/gdrive/MyDrive/Kaggle')

In [ ]:
path = Path.cwd()/"homesite_competition_data"
path.mkdir(exist_ok=True)
Path.BASE_PATH = path
api.competition_download_cli('homesite-quote-conversion', path=path)
file_extract(path/"homesite-quote-conversion.zip")
file_extract(path/"train.csv.zip")
file_extract(path/"test.csv.zip")
path.ls()

homesite-quote-conversion.zip: Skipping, found more recently modified local copy (use --force to force download)


(#6) [Path('homesite-quote-conversion.zip'),Path('sample_submission.csv.zip'),Path('test.csv.zip'),Path('train.csv.zip'),Path('train.csv'),Path('test.csv')]

Settings

In [ ]:
test_size = 0.3
y_block=CategoryBlock()
# n_estimators = [50, 100, 150, 200]
# max_depth = [2, 4, 6, 8]
n_estimators_range = range(50,500,50)
n_estimators_range1 = range(500,1050,50)
n_estimators_range2 = range(1050,1500,50)
n_estimators_range3 = range(1550,2050,50)
max_depth_range = range(1, 12, 2)
max_depth_range1 = range(13,22,2)
max_depth_range2 = range(23,32,2)
learning_rate_range = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
subsample_range = [0.1,0.3,0.5,0.7,1.0]
sampling_methods = ['uniform','gradient_based']
random_seed =42
n_splits = 10 # 10 folds
scoring = "roc_auc"
category_threshold = 20

In [ ]:
from sklearn.metrics import roc_auc_score
# valid_score = roc_auc_score(to_np(targs), to_np(preds[:,1]))
# valid_score

## The [GridsearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) functions

Customising functions from [this page](https://machinelearningmastery.com/tune-number-size-decision-trees-xgboost-python/) and [this page](https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/)to work with our dataset

In [ ]:
import xgboost as xgb
import matplotlib
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot

This function will tune for `n_estimators` only

**Notes**
- Used https://scikit-learn.org/stable/modules/model_evaluation.html#scoring to find correct `scoring` string
- Reading more on [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) to understand if I used the `n_splits` correctly, I chose 10 based on [this article](https://machinelearningmastery.com/k-fold-cross-validation/)

In [ ]:
# XGBoost, Tune n_estimators

def xgboost_tune_estimators(X, y, n_estimators_range, n_splits, sampling_methods, random_seed, scoring):
    matplotlib.use('Agg')
    label_encoded_y = LabelEncoder().fit_transform(y)
    # grid search
    model = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0, verbosity=2, sampling_methods=sampling_methods)
    param_grid = dict(n_estimators=n_estimators_range)
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    grid_search = GridSearchCV(model, param_grid, scoring=scoring, n_jobs=-1, cv=kfold, verbose=4)
    grid_result = grid_search.fit(X, label_encoded_y)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    # plot
    pyplot.errorbar(n_estimators_range, means, yerr=stds)
    pyplot.title("XGBoost n_estimators vs Log Loss")
    pyplot.xlabel('n_estimators')
    pyplot.ylabel('Log Loss')
    pyplot.savefig('n_estimators.png')

This function will tune for `max_depth` value only

In [ ]:
# XGBoost, Tune max_depth
def xgboost_tune_max_depth(X,y, max_depth_range, n_splits, sampling_methods, random_seed, scoring):
    matplotlib.use('Agg')
    # encode string class values as integers
    label_encoded_y = LabelEncoder().fit_transform(y)
    # grid search
    model = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0, verbosity=2, sampling_methods=sampling_methods)
    print(max_depth_range)
    param_grid = dict(max_depth=max_depth_range)
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    grid_search = GridSearchCV(model, param_grid, scoring=scoring, n_jobs=-1, cv=kfold, verbose=4)
    grid_result = grid_search.fit(X, label_encoded_y)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    # plot
    pyplot.errorbar(max_depth_range, means, yerr=stds)
    pyplot.title("XGBoost max_depth vs Log Loss")
    pyplot.xlabel('max_depth')
    pyplot.ylabel('Log Loss')
    pyplot.savefig('max_depth.png')

This function will tune for `learning_rate`

In [ ]:
def xgboost_tune_lr(X, y, lr_range, n_splits, sampling_methods, random_seed, scoring):
    matplotlib.use('Agg')
    label_encoded_y = LabelEncoder().fit_transform(y)
    # grid search
    model = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0, verbosity=2, sampling_methods=sampling_methods)
    param_grid = dict(learning_rate=lr_range)
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    grid_search = GridSearchCV(model, param_grid, scoring=scoring, n_jobs=-1, cv=kfold, verbose=4)
    grid_result = grid_search.fit(X, label_encoded_y)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    # plot
    pyplot.errorbar(lr_range, means, yerr=stds)
    pyplot.title("XGBoost learning_rate vs Log Loss")
    pyplot.xlabel('learning_rate')
    pyplot.ylabel('Log Loss')
    pyplot.savefig('learning_rate.png')

This function will tune for `n_estimators`, `max_depth`, `learning_rate` in combination (takes really long to run)

In [ ]:
# XGBoost, Tune n_estimators, max_depth, learning_rate
def xgboost_tune_n_estimators_and_max_depth_and_lr(X, y, n_estimators_range, max_depth_range, lr_range, n_splits, sampling_methods, random_seed, scoring):
    matplotlib.use('Agg')
    # encode string class values as integers
    label_encoded_y = LabelEncoder().fit_transform(y)
    # grid search
    model = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0, verbosity=2, sampling_methods=sampling_methods)
    print(n_estimators_range)
    print(max_depth_range)
    print(lr_range)
    param_grid = dict(max_depth=max_depth_range, n_estimators=n_estimators_range, learning_rate=lr_range)
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    grid_search = GridSearchCV(model, param_grid, scoring=scoring, n_jobs=-1, cv=kfold, verbose=4)
    grid_result = grid_search.fit(X, label_encoded_y)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    # plot results
    # scores = np.array(means).reshape(len(max_depth_range), len(n_estimators_range))
    # for i, value in enumerate(max_depth_range):
    #   pyplot.plot(n_estimators_range, scores[i], label='depth: ' + str(value))
    # pyplot.legend()
    # pyplot.xlabel('n_estimators')
    # pyplot.ylabel('Log Loss')
    # pyplot.savefig('n_estimators_vs_max_depth.png')

### My useful functions

In [ ]:
def reassign_to_categorical(field, df, y_names, continuous, categorical):
  if ((df[field].isna().sum()==0) and (field not in y_names)):
    field_categories = df[field].unique()
    df[field] = df[field].astype('category')
    df[field].cat.set_categories(field_categories, inplace=True)
    if field in continuous: continuous.remove(field)
    if field not in categorical: categorical.append(field)

  return df, continuous, categorical

In [ ]:
def categorize( df, y_names, cont_names, cat_names, category_threshold=50):
  for field in df.columns:
    if ((len(df[field].unique()) <= category_threshold) and (type(df[field].dtype) != pd.core.dtypes.dtypes.CategoricalDtype)):
      reassign_to_categorical(field, df, y_names, cont_names, cat_names)
  return df, cont_names, cat_names

In [ ]:
def homesite_prep(df_train, df_test, y_names, category_threshold):
    df_train.QuoteConversion_Flag = df_train.QuoteConversion_Flag.astype(dtype='boolean')
    df_train = df_train.set_index('QuoteNumber')
    df_test = df_test.set_index('QuoteNumber')
    df_train['Original_Quote_Date'] = pd.to_datetime(df_train['Original_Quote_Date'])
    df_test['Original_Quote_Date'] = pd.to_datetime(df_test['Original_Quote_Date'])
    df_train = add_datepart(df_train, 'Original_Quote_Date')
    df_test = add_datepart(df_test, 'Original_Quote_Date')
    cont_names, cat_names = cont_cat_split(df_train, dep_var=y_names)
    df_train, cont_names, cat_names = categorize(df_train, y_names, cont_names, cat_names, category_threshold)
    return df_train, df_test, cont_names, cat_names

In [ ]:
def find_y_columns(df_train, df_test):
    y_columns = df_train.columns.difference(df_test.columns)
    return y_columns

### Load the data

In [ ]:
df_train = pd.read_csv(path/"train.csv", low_memory=False)
df_train.head(2)

,QuoteNumber,Original_Quote_Date,QuoteConversion_Flag,Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,CoverageField8,CoverageField9,CoverageField11A,CoverageField11B,SalesField1A,SalesField1B,SalesField2A,SalesField2B,SalesField3,SalesField4,SalesField5,SalesField6,SalesField7,SalesField8,SalesField9,SalesField10,SalesField11,SalesField12,...,GeographicField44A,GeographicField44B,GeographicField45A,GeographicField45B,GeographicField46A,GeographicField46B,GeographicField47A,GeographicField47B,GeographicField48A,GeographicField48B,GeographicField49A,GeographicField49B,GeographicField50A,GeographicField50B,GeographicField51A,GeographicField51B,GeographicField52A,GeographicField52B,GeographicField53A,GeographicField53B,GeographicField54A,GeographicField54B,GeographicField55A,GeographicField55B,GeographicField56A,GeographicField56B,GeographicField57A,GeographicField57B,GeographicField58A,GeographicField58B,GeographicField59A,GeographicField59B,GeographicField60A,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64
0,1,2013-08-16,0,B,23,0.9403,0.0006,965,1.0200,N,17,23,17,23,15,22,16,22,13,22,13,23,T,D,2,1,7,18,3,8,0,5,5,24,V,48649,0,0,0,0,...,8,4,20,22,10,8,6,5,15,13,19,18,16,14,21,23,21,23,16,11,22,24,7,14,-1,17,15,17,14,18,9,9,-1,8,-1,18,-1,10,N,CA
1,2,2014-04-22,0,F,7,1.0006,0.0040,548,1.2433,N,6,8,6,8,5,7,5,8,13,22,13,23,T,E,5,9,5,14,6,18,1,5,5,11,P,26778,0,0,1,1,...,23,24,11,15,21,24,6,11,21,21,18,15,20,20,13,12,12,12,15,9,13,11,11,20,-1,9,18,21,8,7,10,10,-1,11,-1,17,-1,20,N,NJ


In [ ]:
df_test = pd.read_csv(path/"test.csv", low_memory=False)
df_test.head(2)

,QuoteNumber,Original_Quote_Date,Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,CoverageField8,CoverageField9,CoverageField11A,CoverageField11B,SalesField1A,SalesField1B,SalesField2A,SalesField2B,SalesField3,SalesField4,SalesField5,SalesField6,SalesField7,SalesField8,SalesField9,SalesField10,SalesField11,SalesField12,SalesField13,...,GeographicField44A,GeographicField44B,GeographicField45A,GeographicField45B,GeographicField46A,GeographicField46B,GeographicField47A,GeographicField47B,GeographicField48A,GeographicField48B,GeographicField49A,GeographicField49B,GeographicField50A,GeographicField50B,GeographicField51A,GeographicField51B,GeographicField52A,GeographicField52B,GeographicField53A,GeographicField53B,GeographicField54A,GeographicField54B,GeographicField55A,GeographicField55B,GeographicField56A,GeographicField56B,GeographicField57A,GeographicField57B,GeographicField58A,GeographicField58B,GeographicField59A,GeographicField59B,GeographicField60A,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64
0,3,2014-08-12,E,16,0.9364,0.0006,"1,487",1.3045,N,4,4,4,4,3,3,3,4,13,22,13,23,Y,K,13,22,6,16,9,21,0,5,5,11,P,67052,0,0,0,0,0,...,22,23,9,12,25,25,6,9,4,2,16,12,20,20,2,2,2,1,1,1,10,7,25,25,-1,19,19,22,12,15,1,1,-1,1,-1,20,-1,25,Y,IL
1,5,2013-09-07,F,11,0.9919,0.0038,564,1.1886,N,8,14,8,14,7,12,8,13,13,22,13,23,T,E,4,5,3,6,3,6,1,5,5,4,R,27288,1,0,0,0,0,...,23,24,12,21,23,25,7,11,16,14,13,6,17,15,7,5,7,5,13,7,14,14,7,14,-1,4,1,1,5,3,10,10,-1,5,-1,5,-1,21,N,NJ


In [ ]:
y_names = find_y_columns(df_train, df_test)[0]
df_train, df_test, cont_names, cat_names = homesite_prep(df_train, df_test, y_names, category_threshold)

In [ ]:
procs = [Categorify, FillMissing, Normalize]
splits = TrainTestSplitter(test_size=test_size, stratify=df_train[y_names])(df_train)

In [ ]:
to = TabularPandas(df=df_train, procs=procs, cat_names=cat_names, 
                   cont_names=cont_names, y_names=y_names,splits=splits,
                  y_block=y_block)

In [ ]:
sampling_methods[0], sampling_methods[1]

('uniform', 'gradient_based')

## Exploring single parameter tuning performance

### n_estimators

In [ ]:
%time xgboost_tune_estimators(to.xs, to.ys.values.ravel(), n_estimators_range, n_splits, sampling_methods[1], random_seed, scoring)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   30.4s
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  5.1min finished


Best: 0.964941 using {'n_estimators': 450}
0.953037 (0.000664) with: {'n_estimators': 50}
0.957954 (0.000668) with: {'n_estimators': 100}
0.960359 (0.000591) with: {'n_estimators': 150}
0.961898 (0.000540) with: {'n_estimators': 200}
0.962885 (0.000550) with: {'n_estimators': 250}
0.963609 (0.000563) with: {'n_estimators': 300}
0.964180 (0.000594) with: {'n_estimators': 350}
0.964613 (0.000588) with: {'n_estimators': 400}
0.964941 (0.000612) with: {'n_estimators': 450}
CPU times: user 9.27 s, sys: 1.42 s, total: 10.7 s
Wall time: 5min 15s


In [ ]:
%time xgboost_tune_estimators(to.xs, to.ys.values.ravel(), n_estimators_range1, n_splits, sampling_methods[1], random_seed, scoring)

Fitting 10 folds for each of 11 candidates, totalling 110 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 110 out of 110 | elapsed: 25.7min finished


Best: 0.966545 using {'n_estimators': 1000}
0.965281 (0.000917) with: {'n_estimators': 500}
0.965500 (0.000936) with: {'n_estimators': 550}
0.965691 (0.000945) with: {'n_estimators': 600}
0.965869 (0.000955) with: {'n_estimators': 650}
0.966008 (0.000901) with: {'n_estimators': 700}
0.966127 (0.000916) with: {'n_estimators': 750}
0.966237 (0.000894) with: {'n_estimators': 800}
0.966322 (0.000882) with: {'n_estimators': 850}
0.966377 (0.000877) with: {'n_estimators': 900}
0.966441 (0.000879) with: {'n_estimators': 950}
0.966545 (0.000882) with: {'n_estimators': 1000}
CPU times: user 26.1 s, sys: 1.63 s, total: 27.8 s
Wall time: 25min 59s


In [ ]:
%time xgboost_tune_estimators(to.xs, to.ys.values.ravel(), n_estimators_range2, n_splits, sampling_methods[1], random_seed, scoring)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 23.2min finished


Best: 0.966838 using {'n_estimators': 1400}
0.966547 (0.000732) with: {'n_estimators': 1050}
0.966608 (0.000750) with: {'n_estimators': 1100}
0.966677 (0.000760) with: {'n_estimators': 1150}
0.966705 (0.000746) with: {'n_estimators': 1200}
0.966746 (0.000737) with: {'n_estimators': 1250}
0.966796 (0.000750) with: {'n_estimators': 1300}
0.966814 (0.000762) with: {'n_estimators': 1350}
0.966838 (0.000758) with: {'n_estimators': 1400}
0.966838 (0.000732) with: {'n_estimators': 1450}
CPU times: user 25.6 s, sys: 2.16 s, total: 27.8 s
Wall time: 23min 31s


In [ ]:
%time xgboost_tune_estimators(to.xs, to.ys.values.ravel(), n_estimators_range3, n_splits, sampling_methods[1], random_seed, scoring)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  6.2min
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 32.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 36.4min finished


Best: 0.966902 using {'n_estimators': 1950}
0.966850 (0.000720) with: {'n_estimators': 1550}
0.966867 (0.000728) with: {'n_estimators': 1600}
0.966880 (0.000725) with: {'n_estimators': 1650}
0.966870 (0.000727) with: {'n_estimators': 1700}
0.966897 (0.000728) with: {'n_estimators': 1750}
0.966884 (0.000728) with: {'n_estimators': 1800}
0.966889 (0.000721) with: {'n_estimators': 1850}
0.966898 (0.000732) with: {'n_estimators': 1900}
0.966902 (0.000735) with: {'n_estimators': 1950}
0.966889 (0.000737) with: {'n_estimators': 2000}
CPU times: user 33.2 s, sys: 2.44 s, total: 35.6 s
Wall time: 36min 46s


From the batches we got from `0.964941 (0.000612) with: {'n_estimators': 450}` to `0.966902 (0.000735) with: {'n_estimators': 1950}` just tuning `n_estimators`

### max_depth

In [ ]:
%time xgboost_tune_max_depth(to.xs,to.ys.values.ravel(), max_depth_range, n_splits, sampling_methods[1], random_seed, scoring)

range(1, 12, 2)
Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   31.8s
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.2min finished


Best: 0.965565 using {'max_depth': 11}
0.940100 (0.001024) with: {'max_depth': 1}
0.957954 (0.000668) with: {'max_depth': 3}
0.962320 (0.000596) with: {'max_depth': 5}
0.964306 (0.000669) with: {'max_depth': 7}
0.965231 (0.000838) with: {'max_depth': 9}
0.965565 (0.000880) with: {'max_depth': 11}
CPU times: user 6.8 s, sys: 1.42 s, total: 8.22 s
Wall time: 3min 18s


In [ ]:
%time xgboost_tune_max_depth(to.xs,to.ys.values.ravel(), max_depth_range1, n_splits, sampling_methods[1], random_seed, scoring)

range(13, 22, 2)
Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 26.6min finished


Best: 0.965149 using {'max_depth': 13}
0.965149 (0.000994) with: {'max_depth': 13}
0.964474 (0.000903) with: {'max_depth': 15}
0.964005 (0.000966) with: {'max_depth': 17}
0.963855 (0.000978) with: {'max_depth': 19}
0.963873 (0.000889) with: {'max_depth': 21}
CPU times: user 14.4 s, sys: 1.82 s, total: 16.3 s
Wall time: 26min 44s


With our two range checks, it seems like our best results come with `0.965565 (0.000880) with: {'max_depth': 11}` and seems to get worse after that, since `0.965149 (0.000994) with: {'max_depth': 13}` in the 2nd range checked is less than the best in the initial range chechked, and progressively gets worse. So we should do our final fine tuning of permutations of variables only with the first range for `max_depth` and skip tuning with the final range of max_depth

In [ ]:
# %time xgboost_tune_max_depth(to.xs,to.ys.values.ravel(), max_depth_range2, n_splits, sampling_methods[1], random_seed, scoring)

### learning_rate

In [ ]:
%time xgboost_tune_lr(to.xs,to.ys.values.ravel(), learning_rate_range, n_splits, sampling_methods[1], random_seed, scoring)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   41.0s
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.8min finished


Best: 0.963524 using {'learning_rate': 0.3}
0.870361 (0.002025) with: {'learning_rate': 0.0001}
0.872141 (0.003595) with: {'learning_rate': 0.001}
0.906749 (0.001726) with: {'learning_rate': 0.01}
0.957872 (0.001386) with: {'learning_rate': 0.1}
0.962143 (0.001049) with: {'learning_rate': 0.2}
0.963524 (0.001038) with: {'learning_rate': 0.3}
CPU times: user 4.03 s, sys: 1.58 s, total: 5.61 s
Wall time: 1min 54s


The best learning rate, if just tuning that alone is `Best: 0.963524 using {'learning_rate': 0.3}`

In [ ]:
n_estimators_range[0:1], n_estimators_range[1:4], n_estimators_range[4:8]

(range(50, 100, 50), range(100, 250, 50), range(250, 450, 50))

In [ ]:
n_estimators_agg_range = (n_estimators_range[0], n_estimators_range3[-1], 50)
n_estimators_agg_range

(50, 2000, 50)

## Hypothesis: Multi-parameter vs Single parameter tuning
> Test if tuning can lead to lower values and as good as or better metrics than single-parameter tuning alone

Hypothesis: We don't need as large a number of `n_estimators` to get at least as good as a validation metric as when we had a large value for `n_estimtors`
Need to split up the ranges more to not timeout the run in Kaggle. Will stop 1 range after we get a validation metric equal to the `n_estimators=1950` value

In [ ]:
%time xgboost_tune_n_estimators_and_max_depth_and_lr(to.train.xs, to.train.ys.values.ravel(), n_estimators_range[0:1], max_depth_range, learning_rate_range, n_splits, sampling_methods[1], random_seed, scoring)

range(50, 100, 50)
range(1, 12, 2)
[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 10.0min finished


Best: 0.964010 using {'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 50}
0.751979 (0.003046) with: {'learning_rate': 0.0001, 'max_depth': 1, 'n_estimators': 50}
0.869145 (0.002477) with: {'learning_rate': 0.0001, 'max_depth': 3, 'n_estimators': 50}
0.922202 (0.006186) with: {'learning_rate': 0.0001, 'max_depth': 5, 'n_estimators': 50}
0.942591 (0.001473) with: {'learning_rate': 0.0001, 'max_depth': 7, 'n_estimators': 50}
0.950671 (0.001595) with: {'learning_rate': 0.0001, 'max_depth': 9, 'n_estimators': 50}
0.953800 (0.001254) with: {'learning_rate': 0.0001, 'max_depth': 11, 'n_estimators': 50}
0.751979 (0.003046) with: {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 50}
0.869156 (0.002472) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 50}
0.923856 (0.005189) with: {'learning_rate': 0.001, 'max_depth': 5, 'n_estimators': 50}
0.943740 (0.001201) with: {'learning_rate': 0.001, 'max_depth': 7, 'n_estimators': 50}
0.951880 (0.001238) with: {'learning_rate':

`Best: 0.964010 using {'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 50}` to our comparitor score for this range of `0.964941 (0.000612) with: {'n_estimators': 450}` is already pretty close, but still isn't close enough to `0.966902 (0.000735) with: {'n_estimators': 1950}`

In [ ]:
%time xgboost_tune_n_estimators_and_max_depth_and_lr(to.xs, to.ys.values.ravel(), n_estimators_range[1:2], max_depth_range, learning_rate_range, n_splits, sampling_methods[1], random_seed, scoring)

range(100, 150, 50)
range(1, 12, 2)
[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   37.0s
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 21.0min finished


Best: 0.965491 using {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 100}
0.752885 (0.002114) with: {'learning_rate': 0.0001, 'max_depth': 1, 'n_estimators': 100}
0.870359 (0.001553) with: {'learning_rate': 0.0001, 'max_depth': 3, 'n_estimators': 100}
0.926093 (0.002292) with: {'learning_rate': 0.0001, 'max_depth': 5, 'n_estimators': 100}
0.942819 (0.001769) with: {'learning_rate': 0.0001, 'max_depth': 7, 'n_estimators': 100}
0.951621 (0.001413) with: {'learning_rate': 0.0001, 'max_depth': 9, 'n_estimators': 100}
0.954988 (0.001182) with: {'learning_rate': 0.0001, 'max_depth': 11, 'n_estimators': 100}
0.752885 (0.002114) with: {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 100}
0.873178 (0.003716) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100}
0.926634 (0.002313) with: {'learning_rate': 0.001, 'max_depth': 5, 'n_estimators': 100}
0.945112 (0.001885) with: {'learning_rate': 0.001, 'max_depth': 7, 'n_estimators': 100}
0.952877 (0.001527) with: {'lear

Best: `0.965491 using {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 100}`

to our running best of : `Best: 0.964010 using {'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 50}` is a good step up

to our comparitor score for this range of `0.964941 (0.000612) with: {'n_estimators': 450}` is already better

to our individual parameter tuning best of `0.966902 (0.000735) with: {'n_estimators': 1950}` is not as good yet

In [ ]:
%time xgboost_tune_n_estimators_and_max_depth_and_lr(to.xs, to.ys.values.ravel(), n_estimators_range[2:3], max_depth_range, learning_rate_range, n_splits, sampling_methods[1], random_seed, scoring)

range(150, 200, 50)
range(1, 12, 2)
[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   47.6s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 15.7min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 30.9min finished


Best: 0.965929 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 150}
0.752885 (0.002114) with: {'learning_rate': 0.0001, 'max_depth': 1, 'n_estimators': 150}
0.870359 (0.001553) with: {'learning_rate': 0.0001, 'max_depth': 3, 'n_estimators': 150}
0.926181 (0.002251) with: {'learning_rate': 0.0001, 'max_depth': 5, 'n_estimators': 150}
0.943161 (0.002037) with: {'learning_rate': 0.0001, 'max_depth': 7, 'n_estimators': 150}
0.951675 (0.001447) with: {'learning_rate': 0.0001, 'max_depth': 9, 'n_estimators': 150}
0.955047 (0.001206) with: {'learning_rate': 0.0001, 'max_depth': 11, 'n_estimators': 150}
0.752885 (0.002114) with: {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 150}
0.876144 (0.001286) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 150}
0.929654 (0.002302) with: {'learning_rate': 0.001, 'max_depth': 5, 'n_estimators': 150}
0.946549 (0.001667) with: {'learning_rate': 0.001, 'max_depth': 7, 'n_estimators': 150}
0.953203 (0.001327) with: {'lear

Best: `0.965929 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 150}`

to our running best of : `0.965491 using {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 100}` is slightly better

to our comparitor score for this range of `0.964941 (0.000612) with: {'n_estimators': 450}` is better

to our individual parameter tuning best of `0.966902 (0.000735) with: {'n_estimators': 1950}` is not as good yet


In [ ]:
%time xgboost_tune_n_estimators_and_max_depth_and_lr(to.xs, to.ys.values.ravel(), n_estimators_range[3:4], max_depth_range, learning_rate_range, n_splits, sampling_methods[1], random_seed, scoring)

range(200, 250, 50)
range(1, 12, 2)
[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 20.9min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 41.2min finished


Best: 0.966138 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 200}
0.752885 (0.002114) with: {'learning_rate': 0.0001, 'max_depth': 1, 'n_estimators': 200}
0.870359 (0.001553) with: {'learning_rate': 0.0001, 'max_depth': 3, 'n_estimators': 200}
0.926250 (0.002253) with: {'learning_rate': 0.0001, 'max_depth': 5, 'n_estimators': 200}
0.943344 (0.002040) with: {'learning_rate': 0.0001, 'max_depth': 7, 'n_estimators': 200}
0.951746 (0.001453) with: {'learning_rate': 0.0001, 'max_depth': 9, 'n_estimators': 200}
0.955191 (0.001309) with: {'learning_rate': 0.0001, 'max_depth': 11, 'n_estimators': 200}
0.752885 (0.002114) with: {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 200}
0.896939 (0.001870) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 200}
0.933709 (0.003101) with: {'learning_rate': 0.001, 'max_depth': 5, 'n_estimators': 200}
0.947491 (0.001635) with: {'learning_rate': 0.001, 'max_depth': 7, 'n_estimators': 200}
0.953470 (0.001314) with: {'lear

Best: `0.966138 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 200}`

to our running best of : `0.965929 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 150}` is better

to our comparitor score for this range of `0.964941 (0.000612) with: {'n_estimators': 450}` is better

to our individual parameter tuning best of `0.966902 (0.000735) with: {'n_estimators': 1950}` is just slightly worse off

In [ ]:
%time xgboost_tune_n_estimators_and_max_depth_and_lr(to.xs, to.ys.values.ravel(), n_estimators_range[4:5], max_depth_range, learning_rate_range, n_splits, sampling_methods[1], random_seed, scoring)

range(250, 300, 50)
range(1, 12, 2)
[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.0min
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 22.2min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 45.8min finished


Best: 0.966389 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 250}
0.752885 (0.002099) with: {'learning_rate': 0.0001, 'max_depth': 1, 'n_estimators': 250}
0.870360 (0.001705) with: {'learning_rate': 0.0001, 'max_depth': 3, 'n_estimators': 250}
0.926215 (0.001915) with: {'learning_rate': 0.0001, 'max_depth': 5, 'n_estimators': 250}
0.943307 (0.001374) with: {'learning_rate': 0.0001, 'max_depth': 7, 'n_estimators': 250}
0.951765 (0.001050) with: {'learning_rate': 0.0001, 'max_depth': 9, 'n_estimators': 250}
0.954981 (0.001106) with: {'learning_rate': 0.0001, 'max_depth': 11, 'n_estimators': 250}
0.752885 (0.002099) with: {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 250}
0.901628 (0.002865) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 250}
0.935802 (0.001789) with: {'learning_rate': 0.001, 'max_depth': 5, 'n_estimators': 250}
0.947850 (0.001155) with: {'learning_rate': 0.001, 'max_depth': 7, 'n_estimators': 250}
0.953761 (0.001109) with: {'lear

Best: `0.966389 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 250}`

to our running best of : `0.966138 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 200}` is slightly better

to our comparitor score for this range of `0.964941 (0.000612) with: {'n_estimators': 450}` is much better

to our individual parameter tuning best of `0.966902 (0.000735) with: {'n_estimators': 1950}` is not yet better

In [ ]:
%time xgboost_tune_n_estimators_and_max_depth_and_lr(to.xs, to.ys.values.ravel(), n_estimators_range[5:6], max_depth_range, learning_rate_range, n_splits, sampling_methods[1], random_seed, scoring)

range(300, 350, 50)
range(1, 12, 2)
[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 26.5min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 54.5min finished


Best: 0.966422 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300}
0.752885 (0.002099) with: {'learning_rate': 0.0001, 'max_depth': 1, 'n_estimators': 300}
0.870359 (0.001705) with: {'learning_rate': 0.0001, 'max_depth': 3, 'n_estimators': 300}
0.926303 (0.001948) with: {'learning_rate': 0.0001, 'max_depth': 5, 'n_estimators': 300}
0.943667 (0.001428) with: {'learning_rate': 0.0001, 'max_depth': 7, 'n_estimators': 300}
0.951787 (0.001082) with: {'learning_rate': 0.0001, 'max_depth': 9, 'n_estimators': 300}
0.955087 (0.001107) with: {'learning_rate': 0.0001, 'max_depth': 11, 'n_estimators': 300}
0.752885 (0.002099) with: {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 300}
0.903583 (0.001910) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 300}
0.936403 (0.001723) with: {'learning_rate': 0.001, 'max_depth': 5, 'n_estimators': 300}
0.948150 (0.001116) with: {'learning_rate': 0.001, 'max_depth': 7, 'n_estimators': 300}
0.953947 (0.001114) with: {'lear

Best: `0.966422 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300}`

to our running best of : `0.966389 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 250}` is slightly better

to our comparitor score for this range of `0.964941 (0.000612) with: {'n_estimators': 450}` is much better

to our individual parameter tuning best of `0.966902 (0.000735) with: {'n_estimators': 1950}` is not quite better

In [ ]:
%time xgboost_tune_n_estimators_and_max_depth_and_lr(to.xs, to.ys.values.ravel(), n_estimators_range[6:7], max_depth_range, learning_rate_range, n_splits, sampling_methods[1], random_seed, scoring)

range(350, 400, 50)
range(1, 12, 2)
[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.3min
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 30.7min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 63.3min finished


Best: 0.966364 using {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 350}
0.752885 (0.002099) with: {'learning_rate': 0.0001, 'max_depth': 1, 'n_estimators': 350}
0.870359 (0.001705) with: {'learning_rate': 0.0001, 'max_depth': 3, 'n_estimators': 350}
0.926368 (0.001834) with: {'learning_rate': 0.0001, 'max_depth': 5, 'n_estimators': 350}
0.943933 (0.001292) with: {'learning_rate': 0.0001, 'max_depth': 7, 'n_estimators': 350}
0.951854 (0.001094) with: {'learning_rate': 0.0001, 'max_depth': 9, 'n_estimators': 350}
0.955131 (0.001136) with: {'learning_rate': 0.0001, 'max_depth': 11, 'n_estimators': 350}
0.752885 (0.002099) with: {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 350}
0.904082 (0.001852) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 350}
0.937111 (0.001732) with: {'learning_rate': 0.001, 'max_depth': 5, 'n_estimators': 350}
0.948413 (0.001086) with: {'learning_rate': 0.001, 'max_depth': 7, 'n_estimators': 350}
0.954211 (0.001119) with: {'lear

Best: `0.966364 using {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 350}`

to our running best of : `0.966422 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300}` is getting worse

to our comparitor score for this range of `0.964941 (0.000612) with: {'n_estimators': 450}` is much better

to our individual parameter tuning best of `0.966902 (0.000735) with: {'n_estimators': 1950}` is not quite better

In [ ]:
%time xgboost_tune_n_estimators_and_max_depth_and_lr(to.xs, to.ys.values.ravel(), n_estimators_range[7:8], max_depth_range, learning_rate_range, n_splits, sampling_methods[1], random_seed, scoring)

range(400, 450, 50)
range(1, 12, 2)
[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.5min
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 35.0min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 71.9min finished


Best: 0.966493 using {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 400}
0.752885 (0.002099) with: {'learning_rate': 0.0001, 'max_depth': 1, 'n_estimators': 400}
0.870359 (0.001705) with: {'learning_rate': 0.0001, 'max_depth': 3, 'n_estimators': 400}
0.926426 (0.001848) with: {'learning_rate': 0.0001, 'max_depth': 5, 'n_estimators': 400}
0.944073 (0.001318) with: {'learning_rate': 0.0001, 'max_depth': 7, 'n_estimators': 400}
0.951963 (0.001093) with: {'learning_rate': 0.0001, 'max_depth': 9, 'n_estimators': 400}
0.955188 (0.001187) with: {'learning_rate': 0.0001, 'max_depth': 11, 'n_estimators': 400}
0.752885 (0.002099) with: {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 400}
0.904183 (0.001830) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 400}
0.937920 (0.001857) with: {'learning_rate': 0.001, 'max_depth': 5, 'n_estimators': 400}
0.948551 (0.001133) with: {'learning_rate': 0.001, 'max_depth': 7, 'n_estimators': 400}
0.954613 (0.001061) with: {'lear

Best: `0.966493 using {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 400}`

to our running best of : `0.966422 using {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300}` is a very slight improvement

to our comparitor score for this range of `0.964941 (0.000612) with: {'n_estimators': 450}` is better

to our individual parameter tuning best of `0.966902 (0.000735) with: {'n_estimators': 1950}` is not as good

As we can see the most optimal parameters using XGBoost would be to do `n_estimators=300`, `max_depth=5`, `learning_rate=0.2`. Using default values of XGBoost from it's [source code](https://github.com/dmlc/xgboost/blob/master/python-package/xgboost/sklearn.py#L362), Let's run both and compare results side by side

In [29]:
%time xgboost_tune_n_estimators_and_max_depth_and_lr(to.xs, to.ys.values.ravel(), n_estimators_range[8:9], max_depth_range, learning_rate_range, n_splits, sampling_methods[1], random_seed, scoring)

range(450, 500, 50)
range(1, 12, 2)
[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 15.1min
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 39.3min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 80.1min finished


Best: 0.966590 using {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 450}
0.752885 (0.002099) with: {'learning_rate': 0.0001, 'max_depth': 1, 'n_estimators': 450}
0.870359 (0.001705) with: {'learning_rate': 0.0001, 'max_depth': 3, 'n_estimators': 450}
0.926472 (0.001865) with: {'learning_rate': 0.0001, 'max_depth': 5, 'n_estimators': 450}
0.944154 (0.001364) with: {'learning_rate': 0.0001, 'max_depth': 7, 'n_estimators': 450}
0.952004 (0.001115) with: {'learning_rate': 0.0001, 'max_depth': 9, 'n_estimators': 450}
0.955253 (0.001215) with: {'learning_rate': 0.0001, 'max_depth': 11, 'n_estimators': 450}
0.752885 (0.002099) with: {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 450}
0.904500 (0.001786) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 450}
0.938420 (0.001760) with: {'learning_rate': 0.001, 'max_depth': 5, 'n_estimators': 450}
0.948791 (0.001086) with: {'learning_rate': 0.001, 'max_depth': 7, 'n_estimators': 450}
0.954779 (0.001036) with: {'lear

Best: `0.966590 using {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 450}`

to our running best of : `0.966493 using {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 400}`is slightly better

to our comparitor score for this range of `0.964941 (0.000612) with: {'n_estimators': 450}` is better

to our individual parameter tuning best of `0.966902 (0.000735) with: {'n_estimators': 1950}` not quite as good

Ending here for now, as the run times were just getting too long, and we're really really close to comparative performance with the best single-parameter tuned result

## Head to Head Comparisons: Default, vs Recommended vs Max parameter values

In [30]:
n_estimators_original = 100
max_depth_original = 6
learning_rate_original = 0.2

n_estimators_recommended = 450
max_depth_recommended = 5
learning_rate_recommended = 0.1

n_estimators_max = 1950
subsample = 1
enable_categorical=True

In [31]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_valid, y_valid = to.valid.xs, to.valid.ys.values.ravel()

In [33]:
model_original = xgb.XGBClassifier(n_estimators = n_estimators_original, max_depth=max_depth_original, learning_rate=learning_rate_original, subsample=subsample, 
                                  tree_method='gpu_hist', gpu_id=0, verbosity=2, enable_categorical=enable_categorical, sampling_methods=sampling_methods[1])
%time xgb_model_original = model_original.fit(X_train, y_train)
xgb_preds_original = xgb_model_original.predict_proba(X_valid)
xgb_preds_original


CPU times: user 2.24 s, sys: 246 ms, total: 2.49 s
Wall time: 2.47 s


array([[9.9646646e-01, 3.5335247e-03],
       [9.9989974e-01, 1.0023018e-04],
       [7.9425681e-01, 2.0574318e-01],
       ...,
       [9.8356736e-01, 1.6432654e-02],
       [4.6183008e-01, 5.3816992e-01],
       [9.6087682e-01, 3.9123163e-02]], dtype=float32)

In [34]:
model_recommended = xgb.XGBClassifier(n_estimators = n_estimators_recommended, max_depth=max_depth_recommended, learning_rate=learning_rate_recommended, subsample=subsample,
                                    tree_method='gpu_hist', gpu_id=0, verbosity=3, enable_categorical=enable_categorical, sampling_methods=sampling_methods[1])
%time xgb_model_recommended = model_recommended.fit(X_train, y_train)
xgb_preds_recommended = xgb_model_recommended.predict_proba(X_valid)
xgb_preds_recommended

CPU times: user 5.78 s, sys: 224 ms, total: 6.01 s
Wall time: 5.96 s


array([[9.9579751e-01, 4.2025056e-03],
       [9.9995327e-01, 4.6750749e-05],
       [7.4739861e-01, 2.5260136e-01],
       ...,
       [9.8437619e-01, 1.5623793e-02],
       [5.1231825e-01, 4.8768175e-01],
       [9.5093983e-01, 4.9060162e-02]], dtype=float32)

In [36]:
model_max_e = xgb.XGBClassifier(n_estimators = n_estimators_max, max_depth=max_depth_recommended, learning_rate=learning_rate_recommended, subsample=subsample, 
                                  tree_method='gpu_hist', gpu_id=0, verbosity=2, enable_categorical=enable_categorical, sampling_methods=sampling_methods[1])
%time xgb_model_max_e = model_max_e.fit(X_train, y_train)
xgb_preds_max_e = xgb_model_max_e.predict_proba(X_valid)
xgb_preds_max_e

CPU times: user 21.6 s, sys: 315 ms, total: 22 s
Wall time: 21.8 s


array([[9.9950659e-01, 4.9340865e-04],
       [9.9998891e-01, 1.1102343e-05],
       [8.0084145e-01, 1.9915855e-01],
       ...,
       [9.9154103e-01, 8.4589710e-03],
       [5.5457193e-01, 4.4542807e-01],
       [9.7199428e-01, 2.8005721e-02]], dtype=float32)

In [37]:
accuracy(tensor(xgb_preds_original), tensor(y_valid)), accuracy(tensor(xgb_preds_recommended), tensor(y_valid)), accuracy(tensor(xgb_preds_max_e), tensor(y_valid))

(TensorBase(0.9254), TensorBase(0.9271), TensorBase(0.9255))

In [38]:
roc_auc_score(y_score=tensor(xgb_preds_original[:,1:2]), y_true=tensor(y_valid)), roc_auc_score(y_score=tensor(xgb_preds_recommended[:,1:2]), y_true=tensor(y_valid)), roc_auc_score(y_score=tensor(xgb_preds_max_e[:,1:2]), y_true=tensor(y_valid))

(0.9651362820976437, 0.9657186586961237, 0.9643268132563287)

So as we can see, we get a slightly better performance both in accuracy and the `roc_auc_score` (used in Kaggle competition) when we use a lower `n_estimators` and tune together with `max_depth` and `learning_rate` parameters based on the recommendation, than either when we use just the defaults or the maximum value of `n_estimators`